In [ ]:
# 包导入
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import random

In [ ]:
# CNN模块创建
# 步频为1，输入输出的空间维度不变[24,4]
udn_model = keras.models.Sequential([
    keras.layers.Conv2D(128, 2, activation="relu", padding="same",
                         input_shape=[24, 4, 1]),
    keras.layers.Conv2D(64, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(16, 2, activation="relu", padding="same"),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="relu")
])

udn_model.summary()

In [ ]:
# 数据处理
# 验证小区上的流量，标签为小区下一个小时流量
# y_true为小区下个小时的流量
# 数据集为[24,4]的张量，将([i,3]+[i,4])=y_train[]
# X_train[i] --> y_train[i+1]
# 由于tf的reshape只接受32的大小
# 所以用numpy进行数据的构造，并用np.reshap()进行构造
# # 

In [ ]:
# 首个数据写入
hour_rate = [0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2,              0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1,               0.2, 0.3,]
people = 1000
down_flow = 5000.0
up_flow = 100.0
file_path = "datas.tfrecord"

one_day = np.zeros(96)
for i in range(0, 24):
    avg_people = hour_rate[i] * random.randint(30,80) * people / 10 
    max_people = avg_people * random.randint(20, 40) / 10
    down_f = hour_rate[i] * random.randint(30,80) * down_flow / 10
    up_f = hour_rate[i] * random.randint(30, 80) *  up_flow / 10
    one_day[i*4] = avg_people
    one_day[i*4+1] = max_people
    one_day[i*4+2] = up_f
    one_day[i*4+3] = down_f

with tf.io.TFRecordWriter(file_path) as files:
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                "data": tf.train.Feature(
                    float_list=tf.train.Floatlist(value=one_day))
                }))
    files.write(example.SerializePartialToString())

In [43]:
# 读数据
feature_description = {
    "data": tf.io.VarLenFeature(dtype=float)
}

for serialized_example in tf.data.TFRecordDataset(file_path):
    parsed_example = tf.io.parse_single_example(serialized_example,                                                  feature_description)
    print(parsed_example["data"].values)

datasets = parsed_example["data"].values
print(tf.reshape(datasets, [24, 4]))

tf.Tensor(
[  770.  2387.    49.  3200.  1200.  2880.    72.  4300.  1020.  3570.
   168.  7800.   310.   620.    40.  2250.  1600.  4480.   104.  3500.
  2370.  7347.   234.  5100.   330.   693.    60.  3900.  1420.  3976.
    78.  8000.   990.  3564.   114.  5850.   300.  1050.    57.  3900.
   800.  2080.   160.  6000.  1830.  5307.   171.  8100.   370.   999.
    71.  2900.  1260.  3024.   102.  5700.  1650.  3300.   180. 10950.
   410.   984.    79.  3850.  1460.  4234.   120.  5300.  2160.  6480.
   102.  5850.   430.   903.    72.  3550.  1080.  3240.   124.  7900.
  1830.  6588.   234. 10950.   360.  1332.    49.  3650.   960.  3264.
    76.  4000.  2130.  5112.   201.  9150.], shape=(96,), dtype=float32)
tf.Tensor(
[[  770.  2387.    49.  3200.]
 [ 1200.  2880.    72.  4300.]
 [ 1020.  3570.   168.  7800.]
 [  310.   620.    40.  2250.]
 [ 1600.  4480.   104.  3500.]
 [ 2370.  7347.   234.  5100.]
 [  330.   693.    60.  3900.]
 [ 1420.  3976.    78.  8000.]
 [  990.  3564.   

In [ ]:
all_data = tf.constant(["avg_people", "max_people", ])
for i in range(0, 24):
    avg_people = hour_rate[i] * random.randint(3,8) * people 
    max_people = avg_people * random.randint(2, 4)
    down_f = hour_rate[i] * random.randint(3,8) * down_flow
    up_f = hour_rate[i] * random.randint(3, 8) *  up_flow
    update_data = tf.constant([avg_people, max_people, up_f,                                       down_f])

In [ ]:
# 损失函数为负对数似然（negative log likelihood）
def negative_log_likelihood(y_true, y_pred, eps=1e-15):
    p = keras.backend.clip(y_pred, eps, 1-eps)
    loss = keras.backend.sum(-y_true * keras.backend.log(p) -                                    (1-y_true) * keras.backend.log(1-p))
    
    return keras.backend.mean(loss)

In [ ]:
# 编译模块
SGD = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
udn_model.complie(
    loss=negative_log_likelihood,
    optimizer=SGD
)

In [ ]:
# 训练模块
# 在训练时运用mini-batch方法进行小批量梯度、
# 对每个小时进行预测
udn_model.fit(
    x=X_train
    y=y_true
    batch_size=5
    epochs=1
    callbacks="EarlyStopping"
)

In [ ]:
# 验证模块
udn_model.evaluate(

)

In [ ]:
# 进行预测
udn_model.predict(
    
)